<a href="https://colab.research.google.com/github/avinashkarkada/GROMACS-COLAB/blob/main/APO_MD_GROMACS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Make sure you are connected to a GPU node

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
#@title Download and unzip the compressed folder of GROMACS 2020.6 version
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
!tar xzf gromacs.tar.gz

In [ ]:
!pip install cmake --upgrade

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC
pwd
gmx -h

Enter a name for your folder:

In [ ]:
#@title Make a folder in google drive and enter its name Paste your protein.pdb file, force field folder and all mdp files in that folder.


import os
from pathlib import Path 
Folder_name = "" #@param {type:"string"}
mdpath = Path("/content/" + Folder_name + "/") 

if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

os.chdir(mdpath)
!pwd

In [ ]:
#@title Enter the name of the folder in your gdrive below after the path
from google.colab import drive
import os
from pathlib import Path

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Google_Drive_Path = '/content/drive/MyDrive/' #@param {type:"string"}
workDir = Google_Drive_Path


In [ ]:
!cp -r $workDir/* $mdpath
!cp -r $workDir/*.ff $mdpath


In [ ]:
!sudo mv *.pdb protein.pdb
!chmod a+x *
!ls -l

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC
echo "1" | gmx pdb2gmx -f protein.pdb -o protein.gro -water spce -ignh 



In [ ]:
!/content/gromacs/bin/gmx editconf -f protein.gro -o newbox.gro -c -d 0.5 -bt cubic
!/content/gromacs/bin/gmx grompp -f vacuum.mdp -c newbox.gro -p topol.top -o protein-EM-vacuum.tpr -maxwarn 2
!/content/gromacs/bin/gmx mdrun -v -deffnm protein-EM-vacuum -c protein-EM-vacuum.pdb
!/content/gromacs/bin/gmx mdrun -v -deffnm protein-EM-vacuum
!/content/gromacs/bin/gmx solvate -cp protein-EM-vacuum.gro -cs spc216.gro -p topol.top -o solv.gro
!/content/gromacs/bin/gmx grompp -f em.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 2
!/content/gromacs/bin/gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -conc 0.15  -neutral -pname NA -nname CL <<< 13
!/content/gromacs/bin/gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr -maxwarn 2
!/content/gromacs/bin/gmx mdrun -v -deffnm em
!/content/gromacs/bin/gmx grompp -f nvt.mdp -c em.gro -p topol.top -o nvt.tpr -r em.gro -maxwarn 2
!/content/gromacs/bin/gmx mdrun -deffnm nvt -v
!/content/gromacs/bin/gmx grompp -f npt.mdp -c nvt.gro -t nvt.cpt -p topol.top -o npt-1000.tpr -r em.gro -maxwarn 2
!/content/gromacs/bin/gmx mdrun -deffnm npt-1000  -v

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC



arr=(*_*_*_*.itp) && echo "${arr[0]}"

filename=${arr[0]}

if [ -f "${filename}" ]

then

vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E ${arr[0]}
vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E ${arr[1]}
gmx grompp -f npt.mdp -c npt-1000.gro -t npt-1000.cpt -p topol.top -o npt-100.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-100  -v
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E ${arr[0]}
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E ${arr[1]}
gmx grompp -f npt.mdp -c npt-100.gro -t npt-100.cpt -p topol.top -o npt-10.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-10  -v
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E ${arr[0]}
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E ${arr[1]}

else

vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E posre.itp
gmx grompp -f npt.mdp -c npt-1000.gro -t npt-1000.cpt -p topol.top -o npt-100.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-100  -v
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E posre.itp
gmx grompp -f npt.mdp -c npt-100.gro -t npt-100.cpt -p topol.top -o npt-10.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-10  -v
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E posre.itp
fi

gmx grompp -f npt.mdp -c npt-10.gro -t npt-10.cpt -p topol.top -o npt-1.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-1 -v
vim -c ":%s/define/;define/g" -c wq <<< E npt.mdp
gmx grompp -f npt.mdp -c npt-1.gro -t npt-1.cpt -p topol.top -o npt.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt -v
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o protein-10ns.tpr -maxwarn 2


In [ ]:
#@title Production run
!/content/gromacs/bin/gmx mdrun -deffnm protein-10ns -v


In [ ]:
#@title Analysis
%%bash
source /content/gromacs/bin/GMXRC
printf '1\n1' | gmx trjconv -s protein-10ns.tpr -f protein-10ns.xtc -o protein-10ns.pdb -pbc cluster
printf '1\n1' | gmx trjconv -s protein-10ns.tpr -f protein-10ns.xtc -o protein-0-10ns.xtc -pbc cluster
printf '4\n4' | gmx rms -s protein-10ns.tpr -f protein-10ns.pdb -o protein-0-1-RMSD.xvg 
gmx rmsf -s protein-10ns.tpr -f protein-10ns.pdb -o protein-0-10ns-RMSF.xvg -res <<< 3

In [ ]:
#@title Run this to copy the entire project to your drive. This will paste all the results in "result" in your original drive folder.
#Warning!!!!
#Check if you have enough space!
!mkdir $Google_Drive_Path/result
!cp -r $mdpath/* $Google_Drive_Path/result